## CSE545-SDG3-Matrix


In [42]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.types import *
import os

In [43]:
sc = pyspark.SparkContext(appName="SDG3")
sqlCtx = SQLContext(sc)
sc.version

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=SDG3, master=local[*]) created by __init__ at <ipython-input-10-6f706fe7a6d5>:1 

In [44]:
data_path = r'../LinkPE14US'
print(os.path.join(data_path, "VS14LINK.USNUMPUB"))
df = sqlCtx.read.text(os.path.join(data_path, "VS14LINK.USNUMPUB"))  #350 mb
#df = sqlCtx.read.text("/VS14LINK.USDENPUB") # 5gb
#df = sqlCtx.read.text("/*.*PUB") # both
df

../LinkPE14US/VS14LINK.USNUMPUB


DataFrame[value: string]

In [45]:
def blank_as_null(x):
    return when(col(x) != ' ', col(x)).otherwise(np.nan)

In [52]:
# pandas data frame after adding column metadata
pndf = df.select(
    df.value.substr(9,4).cast(IntegerType()).alias('Birth_Year'),
    df.value.substr(13,2).cast(IntegerType()).alias('Birth_Month'),
    df.value.substr(75,2).cast(IntegerType()).alias('Mothers_Age'),
    #df.value.substr(299,3).cast(IntegerType()).alias('Delivery_Weight_lbs'),
    df.value.substr(332,2).cast(IntegerType()).alias('Num_Prev_Cesareans'),
    #435
    df.value.substr(435,1).cast(IntegerType()).alias('Payment_Source'),
    df.value.substr(436,1).cast(IntegerType()).alias('Payment_Recode'),
    df.value.substr(444,2).cast(IntegerType()).alias('Five_Minute_APGAR_Score'),
    df.value.substr(446,1).cast(IntegerType()).alias('Five_Minute_APGAR_Recode'),
    df.value.substr(448,2).cast(IntegerType()).alias('Ten_Minute_APGAR_Score'),
    df.value.substr(450,1).cast(IntegerType()).alias('Ten_Minute_APGAR_Recode'),
    df.value.substr(454,1).cast(IntegerType()).alias('Plurality'),
    df.value.substr(456,1).cast(IntegerType()).alias('Plurality_Imputed'),
    df.value.substr(475,1).cast(StringType()).alias('Sex_of_Infant'),
    df.value.substr(476,1).cast(IntegerType()).alias('Imputed_Sex'),
    df.value.substr(477,2).cast(IntegerType()).alias('Last_Normal_Menses_Month'),
    df.value.substr(481,4).cast(IntegerType()).alias('Last_Normal_Menses_Year'),
    df.value.substr(488,4).cast(IntegerType()).alias('Combined_Gestation_Imputed'),
    df.value.substr(489,1).cast(IntegerType()).alias('Obstetric_Estimate_of_Gestation_Used_Flag'),
    df.value.substr(490,2).cast(IntegerType()).alias('Combined_Gestation_Detail_in_Weeks'),
    df.value.substr(492,1).cast(IntegerType()).alias('Combined_Gestation_Recode_Weeks'),
    df.value.substr(509,2).cast(IntegerType()).alias('Birth_Weight_Recode_14'),
    df.value.substr(511,1).cast(IntegerType()).alias('Birth_Weight_Recode_4'),
    df.value.substr(512,4).cast(IntegerType()).alias('Imputed_Birthwieght'),
    #Abnormal Conditions of the Newborn
    df.value.substr(517,1).cast(StringType()).alias('Assisted_Ventilation'),
    df.value.substr(518,1).cast(StringType()).alias('Assisted_Ventilation_Greater_Than_6_Hours'),
    df.value.substr(519,1).cast(StringType()).alias('Admission_to_NICU'),
    df.value.substr(520,1).cast(StringType()).alias('Surfactant'),
    df.value.substr(521,1).cast(StringType()).alias('Antibiotics'),
    df.value.substr(522,1).cast(StringType()).alias('Seizures'),
    df.value.substr(531,1).cast(IntegerType()).alias('No_Abnormal_Conditions_Checked'),
    #
    #Congenital Anomalies of the Newborn
    df.value.substr(537,1).cast(StringType()).alias('Anencephaly'),
    df.value.substr(538,1).cast(StringType()).alias('Meningomyelocele_or_Spina_Bifida'),
    df.value.substr(539,1).cast(StringType()).alias('Cyanotic_Congenital_Heart_Disease'),
    df.value.substr(540,1).cast(StringType()).alias('Congenital_Diaphragmatic_Hernia'),
    df.value.substr(541,1).cast(StringType()).alias('Omphalocele'),
    df.value.substr(542,1).cast(StringType()).alias('Gastroschisis'),
    df.value.substr(549,1).cast(StringType()).alias('Limb_Reduction_Defect'),
    df.value.substr(550,1).cast(StringType()).alias('Cleft_Lip_w_or_wo_Cleft_Palate'),
    df.value.substr(551,1).cast(StringType()).alias('Cleft_Palate_alone'),
    df.value.substr(553,1).cast(StringType()).alias('Suspected_Chromosomal_Disorder'),
    df.value.substr(554,1).cast(StringType()).alias('Hypospadias'),
    df.value.substr(561,1).cast(IntegerType()).alias('No_Congenital_Anomalies_Checked'),
    #
    df.value.substr(568,1).cast(StringType()).alias('Infant_Living_at_Time_of_Report'),
    df.value.substr(569,1).cast(StringType()).alias('Infant_Being_Breastfed')
    #870
    
).withColumn("id", monotonically_increasing_id())

pndf.printSchema
pndf = pndf.na.fill({'Num_Prev_Cesareans': 0.0})
#pndf = pndf.withColumn("Infant_Living", blank_as_null("Infant_Living"))

In [53]:
from pyspark.ml.feature import StringIndexer
stringIndexer = StringIndexer(inputCol="Infant_Living_at_Time_of_Report", outputCol='Infant_Living_Index')
pndf = stringIndexer.fit(pndf).transform(pndf)
pndf.head(2)

[Row(Birth_Year=2013, Birth_Month=7, Mothers_Age=24, Num_Prev_Cesareans=0, Payment_Source=2, Payment_Recode=2, Five_Minute_APGAR_Score=9, Five_Minute_APGAR_Recode=4, Ten_Minute_APGAR_Score=88, Ten_Minute_APGAR_Recode=5, Plurality=1, Plurality_Imputed=None, Sex_of_Infant='F', Imputed_Sex=None, Last_Normal_Menses_Month=10, Last_Normal_Menses_Year=2012, Combined_Gestation_Imputed=None, Obstetric_Estimate_of_Gestation_Used_Flag=None, Combined_Gestation_Detail_in_Weeks=42, Combined_Gestation_Recode_Weeks=1, Birth_Weight_Recode_14=11, Birth_Weight_Recode_4=3, Imputed_Birthwieght=4119, Assisted_Ventilation='N', Assisted_Ventilation_Greater_Than_6_Hours='N', Admission_to_NICU='N', Surfactant='N', Antibiotics='N', Seizures='N', No_Abnormal_Conditions_Checked=1, Anencephaly='N', Meningomyelocele_or_Spina_Bifida='N', Cyanotic_Congenital_Heart_Disease='N', Congenital_Diaphragmatic_Hernia='N', Omphalocele='N', Gastroschisis='N', Limb_Reduction_Defect='N', Cleft_Lip_w_or_wo_Cleft_Palate='N', Cleft_P

In [55]:
from pyspark.ml.feature import VectorAssembler
pca_cols = pndf.columns
pca_cols.remove('Infant_Living_at_Time_of_Report')
assembler = VectorAssembler(inputCols=pca_cols, outputCol='features')
vector_df = assembler.transform(pndf)
vector_df.head()

IllegalArgumentException: 'Data type StringType is not supported.'

### Scaling and Normalization

In [21]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='features', 
                        outputCol='scaledFeatures', withMean=False, withStd=True) # TODO: should withMean be True??
scaled_df = scaler.fit(vector_df).transform(vector_df)
scaled_df.head()

Row(Birth_Year=2013, Birth_Month=7, Mothers_Age=24, Gestational_Age_weeks=10, Num_Prev_Cesareans=0, Plurality=1, Infant_Living='Y', id=0, Infant_Living_Index=0.0, features=DenseVector([2013.0, 7.0, 24.0, 10.0, 0.0, 1.0, 0.0, 0.0]), scaledFeatures=DenseVector([6292.0563, 2.0564, 3.7862, 1.0951, 0.0, 2.3585, 0.0, 0.0]))

In [22]:
from pyspark.ml.feature import Normalizer
nrmlzer = Normalizer(inputCol='scaledFeatures', outputCol='normalizeFeatures', p=1.0)
l1Normalized = nrmlzer.transform(scaled_df)
l1Normalized.head()

Row(Birth_Year=2013, Birth_Month=7, Mothers_Age=24, Gestational_Age_weeks=10, Num_Prev_Cesareans=0, Plurality=1, Infant_Living='Y', id=0, Infant_Living_Index=0.0, features=DenseVector([2013.0, 7.0, 24.0, 10.0, 0.0, 1.0, 0.0, 0.0]), scaledFeatures=DenseVector([6292.0563, 2.0564, 3.7862, 1.0951, 0.0, 2.3585, 0.0, 0.0]), normalizeFeatures=DenseVector([0.9985, 0.0003, 0.0006, 0.0002, 0.0, 0.0004, 0.0, 0.0]))

### Dimensionality Reduction: PCA

In [23]:
from pyspark.ml.feature import PCA
num_principal_comp = 4
pca = PCA(k=num_principal_comp, inputCol='normalizeFeatures', outputCol='features_pca')
pca_model = pca.fit(l1Normalized)
pca_feat = pca_model.transform(l1Normalized).select('features_pca')
pca_feat.show(truncate=False)

+--------------------------------------------------------------------------------------+
|features_pca                                                                          |
+--------------------------------------------------------------------------------------+
|[-0.9328793047880449,0.023051434768484664,-0.012615119654158995,0.003931802576143753] |
|[-0.9327163038056777,0.023017504060830096,-0.012683599857090907,0.003917862798176857] |
|[-0.9327612315101593,0.022997542762369504,-0.012713401868340616,0.003939066656460864] |
|[-0.9328066253881045,0.023001673448548622,-0.012718016294802991,0.003960496036609716] |
|[-0.9325529774210644,0.023013697442188515,-0.01273538839333199,0.0038361783630625557] |
|[-0.9325858724385955,0.02297695878148644,-0.012744315652816437,0.003876007917191397]  |
|[-0.9328756008287072,0.02295064860334851,-0.012709027018294985,0.004019653845095318]  |
|[-0.9326374299555169,0.022977022983273362,-0.012798300873610504,0.00392007304286202]  |
|[-0.9327455933665315